In [ ]:
import torch
import torchaudio

print(torch.__version__)from google.colab import drive
drive.mount('/content/drive')
print(torchaudio.__version__)

import io
import os
import tarfile
import tempfile

#import boto3
import matplotlib.pyplot as plt
import requests
#from botocore import UNSIGNED
#from botocore.config import Config
from IPython.display import Audio
from torchaudio.utils import download_asset

!pip install modelscope
!pip install funasr

# sample rate
import librosa
import soundfile as sf

# told
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# TOLD 샘플 예제

# initialize pipeline
inference_diar_pipline = pipeline(
    mode="sond_demo",
    num_workers=0,
    task=Tasks.speaker_diarization,
    diar_model_config="sond.yaml",
    model='damo/speech_diarization_sond-zh-cn-alimeeting-16k-n16k4-pytorch',
    reversion="v1.0.5",
    sv_model="damo/speech_xvector_sv-zh-cn-cnceleb-16k-spk3465-pytorch",
    sv_model_revision="v1.2.2",
)

# input: a list of audio in which the first item is a speech recording to detect speakers,
# and the following wav file are used to extract speaker embeddings.
audio_list = [
    "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_data/speaker_diarization/record.wav",
    "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_data/speaker_diarization/spk1.wav",
    "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_data/speaker_diarization/spk2.wav",
    "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_data/speaker_diarization/spk3.wav",
    "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/ASR/test_data/speaker_diarization/spk4.wav",
]

results = inference_diar_pipline(audio_in=audio_list)

# output
print(results)

In [ ]:
# output
'''
{'text': '
spk1 [(0.0, 8.88), (10.56, 15.2)]\n
spk2 [(8.88, 9.76)]\n
spk3 [(9.6, 11.04), (15.12, 15.68)]\n
spk4 [(11.2, 11.76)]'}
'''

### 1. 12월 침착맨 통닭천사 테스트

In [ ]:
## 데이터 EDA

# 기본정보 확인해보기
sample_paper_wav = download_asset("/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/norm_chimtongshorts.wav")
print(torchaudio.info(sample_paper_wav))

# 침통 파일 : AudioMetaData(sample_rate=8000, num_frames=22052, num_channels=1, bits_per_sample=16, encoding=PCM_S)
# sample rate 변환 : AudioMetaData(sample_rate=16000, num_frames=44104, num_channels=1, bits_per_sample=16, encoding=PCM_S)
# 원본_record : AudioMetaData(sample_rate=16000, num_frames=256000, num_channels=1, bits_per_sample=16, encoding=PCM_S)
# 원본_spk2 : AudioMetaData(sample_rate=16000, num_frames=153760, num_channels=1, bits_per_sample=16, encoding=PCM_S)


In [ ]:
# ## 전처리1 : sample rate 변환 -> audicity로 진행

# file_path = '/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/chimtongshorts_chim.wav'
# # WAV 파일 로드 (현재 sample rate: 8000)
# audio_data, sr = librosa.load(file_path, sr=8000)
# # 변경할 샘플 레이트 설정
# desired_sr = 16000
# resampled_data = librosa.resample(y=audio_data,orig_sr=44100, target_sr=16000)
# print("새로운 sample rate:", desired_sr)

# new_file_path = '/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/new_chimtongshorts_chim.wav'
# sf.write(new_file_path, resampled_data, desired_sr)


In [ ]:
## 전처리2 : stereo to mono -> audicity로 진행

In [ ]:
# 침착맨, 통천 입력

audio_list = [
    "/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/norm_chimtongshorts.wav", # 전체
    "/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/norm_chimtongshorts-tong.wav", # 침착맨
    "/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/norm_chimtongshorts_chim.wav", # 통천
]

results = inference_diar_pipline(audio_in=audio_list)
print(results)



In [ ]:
# 시각화

def plot_waveform(waveform, sample_rate):
  waveform_np = waveform.numpy()
  num_channels, num_frames = waveform_np.shape
  figures, axes = plt.subplots(num_channels, 1)
  time_axis = torch.arange(0, num_frames) / sample_rate

  if num_channels==1:
    axes = [axes]

  for c in range(num_channels):
    axes[c].plot(time_axis, waveform[c], linewidth=1)
    axes[c].grid(True)

    if num_channels > 1:
      axes[c].set_ylabel(f'channel {c+1}')

sample_wav = "/content/drive/MyDrive/23-2_kubig_contest/TOLD_test_12_28/norm_chimtongshorts.wav"
waveform, sample_rate = torchaudio.load(sample_wav)
plot_waveform(waveform, sample_rate)